In [5]:
import bokeh
import numpy as np
import pandas as pd
import sqlite3


def db_connect():
    conn = sqlite3.connect('output/sales_new_seed.db')
    return conn

# def get_col_names():
    
#     col_names = []
    
#     with db_connect() as conn:
#         c = conn.cursor()
#         c.execute('PRAGMA table_info(sales)')
#         for col in c.fetchall():
#             col_names.append(col[1])
#     return col_names

# def db_query(query):
    
#     results_dict = {
#         'sale_id': [],
#         'sale_date': [],
#         'store_id': [],
#         'item_id': [],
#         'bottles_sold': [],
#         'sale_value': [],
#         'sale_vol_ml': [],        
#     }    
    
#     with db_connect() as conn:
#         c = conn.cursor()
#         c.execute(query)
        
#         rows = c.fetchall()
#         for row in rows:
#             results_dict['sale_id'].append(row[0])
#             results_dict['sale_date'].append(row[1])
#             results_dict['store_id'].append(row[2])
#             results_dict['item_id'].append(row[3])
#             results_dict['bottles_sold'].append(row[4])
#             results_dict['sale_value'].append(row[5])
#             results_dict['sale_vol_ml'].append(row[6])
            
#         return results_dict
            

# sales_col_names = get_col_names()
# get_x_rows = db_query("SELECT * FROM sales AS s WHERE strftime('%Y', s.sale_date) = '2012'")

# ten_rows_frame = pd.DataFrame(data=get_x_rows)
# ten_rows_frame.sort_values(by=['sale_date']).head()



In [17]:
year_query = "SELECT * FROM sales AS s WHERE strftime('%Y', s.sale_date) = '2012'"

base_query = "SELECT * FROM sales LIMIT 10"

sql_frame = pd.read_sql_query(base_query, db_connect(), parse_dates=['sale_date'])

sql_frame.sort_values(by=['sale_date']).head()

,sale_id,sale_date,store_id,item_id,bottles_sold,sale_value,sale_vol_ml
3,S28867700001,2015-11-04,2513,237,3,16002,5250
6,S28865700001,2015-11-09,2538,238,6,10458,9000
7,S28869500001,2015-11-10,3942,237,2,10668,3500
5,S28869200001,2015-11-11,3650,237,1,5334,1750
2,S29050300001,2015-11-16,3549,249,2,1920,300
